#### Projects MAP 556
### Monte Carlo methods and stochastic processes
##### GNABEYEU MBIADA Emmanuel
##### ALIOUA Imrane 

## Variance Reduction Applied to Machine Learning for Pricing Bermudan/American Options in High Dimension

> We implement here by a backward dynamic programming algorithm, an efficient method to compute the price of multi-asset American options following a Black-Scholes dynamics, based on Machine Learning, Monte Carlo simulations and variance reduction technique. 

> The backward dynamic programming algorithm considers a finite number of uniformly distributed exercise dates. On these dates, the option value is computed as the maximum between the exercise value and the continuation value, which is obtained by means of Gaussian process regression technique and Monte Carlo simulations. 


> We will show that such a method performs well for low dimension baskets but it is not accurate for very high dimension baskets. In order to improve the dimension range and then  overcome the
problem of the curse of dimensionality, we employ the European option price as a control variate, which allows us to treat very large baskets and moreover to reduce the variance of price estimators. 

### Useful Library

In [35]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm 
from scipy.stats import norm as scnorm
#import plotly.graph_objects as go
from ipywidgets import interact, widgets
from scipy.stats import qmc
from scipy import linalg 
import scipy.optimize as opt

from sklearn.datasets import make_friedman2
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel

In [48]:
# Supressing the warning messages
import warnings
warnings.filterwarnings('ignore')

### American options in the multi-dimensional Black-Scholes model
An American option with maturity T is a derivative instrument whose holder can exercise the intrinsic
optionality at any moment, from inception up to maturity. \\
 Let $S = (S_t)$ $t \in [0,T]$ denote the d-dimensional underlying process. Such a stochastic process is assumed to randomly evolve according to the multidimensional Black-Scholes model: 
 
Under the risk neutral measure, such a model is given by the following equation
$dS^i_t = (r − \eta_i) S^i_t dt + \sigma_i S^i_tdW^i_t$ 
, i = 1, . . . , d, 
with $S_0 = (s_{0,1}, . . . , s_{0,d})^⊤\in \mathbb{R}^d_+$ the spot price, r the (constant) spot interest rate, $\eta = (\eta_1, . . . , \eta_d)^⊤ $the
vector of (constant) dividend rates, $\sigma = (\sigma_1, . . . , \sigma_d)^⊤$
the vector of (constant) volatilities,

 W a d-dimensional
correlated Brownian motion and $ρ_{ij}$ the instantaneous correlation coefficient between $W^i_t$
and $W^j_t$. 

Moreover,
let $\Psi(S_T)$ denote the cash-flow associated with the option. Thus, the price at time t of an American option
having maturity T and payoff function $\Psi : \mathbb{R}^d_+ \rightarrow \mathbb{R}$ is then
$ v^{AM}(t, x) = sup_{τ \in \mathbb{T}_{t,T}} \mathbb{E}_{t,x}[e^{−r(τ−t)}Ψ (S_τ )]$

For simulation purposes, the d−dimensional Black-Scholes model can be written alternatively using the Cholesky decomposition. Specifically, for $i \in {1, . . . , d}$ we can write

$dS^i_t =S^i_t( (r − \eta_i)  dt + \sigma_i \Sigma_i dB^i_t)$ 

where B is a d-dimensional Brownian motion and $\Sigma_i$
is the i-th row of the matrix $\Sigma_i $ defined as a square root(Cholesky decomposition)
of the correlation matrix $\Gamma$ given by $\Gamma_{ij}= \rho_{ij}$

#### CoVariance and Correlation Matrix

In [32]:
def variance(sigma_i,d):
    # compute sigma
    return np.array([sigma_i for _ in range(d)])

def sqrtmh(A):
    vals, vecs = linalg.eigh(A)
    return vecs @ np.diag(np.sqrt(vals)) @ vecs.T.conjugate()

def square_root_correlation_matrix(rho,d,sigma_i):
    # compute Gamma and it square gamma
    Gamma=rho*np.ones(d)+(1-rho)*np.eye(d)
    CorrMatrix=(sigma_i**2)*Gamma
    return sqrtmh(Gamma),CorrMatrix

#### Geometric and Arithmetic Basket Put Options and  Maximum Call Option

In [22]:
class BasketOptions:
    def __init__(self, Spot , Strike: float ):
        self.S = Spot
        self.K = Strike
        
    def __Arithmetic_Basket_Put_Options__(self,S,K):
        """ S: array of side (d,1)
        """
        self.__init__(self, S , K )
        return np.maximum(self.K-self.S.mean(),0)
    
    def Call_Maximum_d_assets_American_option(self,S,K):
        """ S: array of side (d,1)
        """
        self.__init__(self, S , K )
        return np.maximum(self.S.max()-self.K,0)
    
    def Geometric_Basket_Put_Options(self,S,K):
        """ S: array of side (d,1)
        """
        self.__init__(self, S , K )
        prod=1
        d=self.S.shape[0]
        for i in range(d):
            prod*=S[i]
            u= K-prod**(1/d)
        return np.maximum(u,0)

    
def Geometric_Basket_Put_Options(S,K):
    """ S: array of side (d,1)
    """
    d=S.shape[0]
    return max(K - np.prod(S)**(1 / d), 0)

def Arithmetic_Basket_Put_Options(S,K):
    """ S: array of side (d,1)
    """
    return np.maximum(K-S.mean(), 0)


def Call_Maximum_d_assets_American_option(S,K):
    """ S: array of side (d,1)
    """
    return np.maximum(np.maximum(S) - K, 0)

## Halton sequence

In [2]:
def next_prime():
    def is_prime(num):
        "Checks if num is a prime value"
        for i in range(2,int(num**0.5)+1):
            if(num % i)==0: return False
        return True

    prime = 3
    while(1):
        if is_prime(prime):
            yield prime
        prime += 2

In [3]:
def vdc(n, base=2):
    vdc, denom = 0, 1
    while n:
        denom *= base
        n, remainder = divmod(n, base)
        vdc += remainder/float(denom)
    return vdc

In [4]:
def halton_sequence(size, dim):
    seq = []
    primeGen = next_prime()
    next(primeGen)
    for d in range(dim):
        base = next(primeGen)
        seq.append([vdc(i, base) for i in range(size)])
    return seq

# Geometric and Arithmetic Basket Put Options.


In [5]:
## Geometric and Arithmetic Basket Put Options.
# Define dimension.
d = 2 #2, 5, 10, 20, 40, 100
# Maturity.
T = 1
# S
Szero = 100
# Strike. 
K = 100
# Returns.
r = 0.05
# Dividend Rates.
eta = 0
# Standard Deviation.
dev_std = 0.2
# Correlations
rho = 0.2
# Number of exercise dates.
N = 10
# Number of points.
P = 250 #250, 500, 1000
# Number of Monte Carlo simulations.
M = 10**3 #10**3, 10**4, 10**5
# Points for the computation of the European prices with GRP-EI
Q = 10000
# Halton Process.
sample = np.array(halton_sequence(Q, d))


# Number of time steps.
delta_t = T/N
# Discrete exercise dates.
t = np.array([i * delta_t for i in range(1,N+1)])

G_n_p_m = np.array([[[np.random.normal(size = d) for _ in range(M)] for _ in range(P)] for _ in range(N)])
gamma_corr = np.eye(d) * (1-rho) + np.ones((d,d)) * rho
sigma_maj_mat = np.sqrt(gamma_corr) ## Sigma majuscule
pi_matrix = np.ones((d,d)) * rho * dev_std 



# Call on the Maximum option.

In [6]:
## Call on the Maximum option.
# Dimension.
d = 2 #2, 5, 10, 20, 30, 50, 100
# Maturity.
T = 3
# S
Szero = 100
# Strike. 
K = 100
# Returns.
r = 0.05
# Dividend Rates.
eta = 0.1
# Standard Deviation.
dev_std = 0.2
# Correlations
rho = 0
# Number of exercise dates.
N = 9 #9
# Number of points.
P = 250 #250, 500, 1000
# Number of Monte Carlo simulations.
M = 10 #10**3, 10**4, 10**5


# Number of time steps.
delta_t = T/N
# Discrete exercise dates.
t = np.array([i * delta_t for i in range(1,N+1)])

G_n_p_m = np.array([[[np.random.normal(size = d) for _ in range(M)] for _ in range(P)] for _ in range(N)])
gamma_corr = np.eye(d) * (1-rho) + np.ones((d,d)) * rho
sigma_maj_mat = np.sqrt(gamma_corr) ## Sigma majuscule
pi_matrix = np.ones((d,d)) * rho * dev_std 

## Preprocessing

In [7]:
## Global variables.
x = np.array([[[Szero * np.exp((r-eta-0.5*dev_std**2)*t[n] + np.exp(np.sqrt(t[n]) * dev_std * np.dot(sigma_maj_mat, norm.ppf(sample[:,p])))) for i in range(d)] for p in range(P)] for n in range(N)])
x_tilde = np.array([[[x[n][p] * np.exp((r-eta-0.5*dev_std**2)*delta_t + np.exp(np.sqrt(delta_t) * dev_std * np.dot(sigma_maj_mat, norm.ppf(sample[:,p])) * G_n_p_m[n][p][m])) for m in range(M)] for p in range(P)] for n in range(N) ])

In [10]:
sigma_f = 1
l = 1
sigma_p = 1

In [12]:
def pricing_EU_option(z, z_tilde, t, t_tilde, weight, sigma_f = sigma_f, l = l):
  v_eu = np.exp(-r*(T-t)) * (sigma_f ** 2) * l**d / np.sqrt(np.linalg.det(np.dot(T-t, pi_matrix) + l**2 * np.eye(d))) 
  return v_eu * np.sum(weight*np.exp(-0.5 * np.dot((z-z_tilde), np.dot(np.linalg.inv(np.dot((T-t_tilde), pi_matrix + l**2 * np.eye(d))),z-z_tilde))))

## Kernel Function

In [13]:
def kernel_function(x, y, sigma_f=1, l=1):
    kernel = sigma_f * np.exp(- (np.linalg.norm(x - y)**2) / (2 * l**2))
    return kernel

## Covariance Matrices

In [14]:
import itertools

def compute_cov_matrices(x, x_star, sigma_f=1, l=1):
    """
    Compute components of the covariance matrix of the joint distribution.
    
    We follow the notation:
    
        - K = K(X, X) 
        - K_star = K(X_*, X)
        - K_star2 = K(X_*, X_*)
    """
    d = x.shape[0]
    d_star = x_star.shape[0]

    K = [kernel_function(i, j, sigma_f=sigma_f, l=l) for (i, j) in itertools.product(x, x)]

    K = np.array(K).reshape(d, d)
    
    K_star2 = [kernel_function(i, j, sigma_f=sigma_f, l=l) for (i, j) in itertools.product(x_star, x_star)]

    K_star2 = np.array(K_star2).reshape(d_star, d_star)
    
    K_star = [kernel_function(i, j, sigma_f=sigma_f, l=l) for (i, j) in itertools.product(x_star, x)]

    K_star = np.array(K_star).reshape(d_star, d)
    
    return (K, K_star2, K_star)

## Find hyperparameters of the Kernel
#### Optimisation of the Log-Likedlihood

The parameters $\sigma_f^2, \sigma_l$ of the kernel function and $\sigma_P^2$
of the noise are called hyperparameters and need to be estimated. A
common approach is to consider the maximum likelihood estimates which can be obtained by maximizing
the log-likelihood function(or minimizing the negative log-likelihood function) of the training data.

>The minimize() function is shorthand for scipy.optimize.minimize(). This function returns a dictionary of objects including the solution to the optimization problem and whether the problem actually solved. The minimize function has three mandatory arguments, plus a lot of options. You can experiment with the options on the minimize() documentation page. 

>1. The first argument of the minimize function is the criterion function (crit() in this example) from which the minimize() function will test values of the parameters in searching for the minimum value.
>2. The second argument is an initial guess for the values of the parameters that minimize the criterion function crit().
>3. The third argument is the tuple of all the objects needed to solve the criterion function in crit().

In [15]:
def Compute_K(X, sigma_f, sigma_l):
    '''
    --------------------------------------------------------------------
    This function computes the kernel matrix
    --------------------------------------------------------------------
    INPUTS:
    X: matrix, 
    sigma_f,sigma_l: Kernel parameters

    RETURNS: K(X,X)
    --------------------------------------------------------------------
    '''
    n = X.shape[0]
    K = [kernel_function(X, X, sigma_f=sigma_f, l=sigma_l) for (i, j) in itertools.product(X, X)]
    K = np.array(K).reshape(n, n)
    
    return K

def log_lik_norm(X,y,sigma_f,sigma_l,sigma_p):
    '''
    --------------------------------------------------------------------
    This function computes the negative of the log likelihood function
    given parameters and data.
    --------------------------------------------------------------------
    INPUTS:
    INPUTS:
    X: matrix, y: vector 
    sigma_f,sigma_l: Kernel parameters
    sigma_p: noise
    RETURNS: neg_log_lik_val
    --------------------------------------------------------------------
    '''
    K= Compute_K(X,sigma_f,sigma_l) 
    p=K.shape[0]
    A=K + (sigma_p**2)*np.identity(p)
    # assert(A.shape==p)
    log_lik_val =0.5*(np.log(np.linalg.det(A))+ y.T@(np.linalg.inv(A))@ y)
    neg_log_lik_val = -log_lik_val
    return neg_log_lik_val

def crit(params,X,y): #, *args):
    '''
    --------------------------------------------------------------------
    This function computes the negative of the log likelihood function
    given parameters and data. This is the minimization problem version
    of the maximum likelihood optimization problem
    --------------------------------------------------------------------
    INPUTS:
    params = (3,) vector, ([sigma_f,sigma_l,sigma_p])
    args   = length 2 tuple, (xvals, cutoff)

    RETURNS: neg_log_lik_val
    --------------------------------------------------------------------
    '''
    # X,y =args
    sigma_f,sigma_l,sigma_p = params
    
    # y = args

    neg_log_lik_val = log_lik_norm(X,y,sigma_f,sigma_l,sigma_p)
    
    return neg_log_lik_val

#### Function to compute the Hyperparameters

In [17]:
def ComputeSigma_lfp(X,y):
    '''
    --------------------------------------------------------------------
    This function computes by optimization the hyperparameters of the GPR model.
    --------------------------------------------------------------------
    INPUTS:
    X: matrix, y: vector 
    RETURNS: sigma_f_MLE, sigma_l_MLE,sigma_p_MLE
    --------------------------------------------------------------------
    '''
    # resee the initialisation: avoid local minimum
    sigma_l_init = np.random.uniform(0, 1)  
    sigma_f_init=  np.random.uniform(0, 1)
    sigma_p_init = np.random.uniform(0, 1)
    mle_args = (X,y)
    results = opt.minimize(crit, params_init, args=mle_args)
    sigma_f_MLE, sigma_l_MLE,sigma_p_MLE  = results.x
    return sigma_f_MLE, sigma_l_MLE,sigma_p_MLE

##  The GPR Monte Carlo Method

>Let us introduce the GPR Monte Carlo approach. We approximate the price of an American option with the price of a Bermudan option on the same basket. Specifically, let N be the number of time steps and $\Delta t = \frac{T}{N}$ the time increment. 

>The discrete exercise dates are $t_n = n \Delta t $, as n = 1, . . . , N. If x represents the vector of the underlying prices at the exercise date $t_n$, then the price of the Bermudan option $v^{BM}$ is given by: $ v^{BM}(t_n,x) = max(\Psi(x), \mathbb{E}_{t_n,x}[e^{−r∆t}v^{BM}(t_{n+1}, S_{t_{n+1}})])$ 

By knowing the function $v^{BM}(t_{n+1}, ·)$, one can compute $v^{BM}(t_{n}, ·)$ by approximation of the
expectation in the above expression.

>In order to do that, we consider a set $X^n$ of P points whose coordinates represent
certain possible values for the underlyings at time $t_n$:
$:
X^n =\{ {x^{n,p} = (x^{n,p}_1
, . . . , x^{n,p}_d), p = 1, . . . , P} \}\subset \mathbb{R}^d$

>For each $x^{n,p} \in X^n$ $v^{BM}(t_{n}, ·) $ is compute by means
of a one step Monte Carlo simulation, by simulation a new set
$ \widetilde{X}^{n,p} =\{ {\widetilde{x}^{n,p,m} = (\widetilde{x}^{n,p,m}_1
, . . . , \widetilde{x}^{n,p,m}_d), m = 1, . . . , M} \}\subset \mathbb{R}^d $

Then, the option price can be approximated for each $x^{n,p} \in X^n$ by: $ \hat{v}^{BM}(t_n,x^{n,p}) = max(\Psi(x^{n,p}), \frac{e^{−r∆t}}{M} \sum_{m=1}^M v^{BM}(t_{n+1}, \widetilde{x}^{n,p,m}))$ 

Of course if the quantities $v^{BM}(t_{n+1}, \widetilde{x}^{n,p,m})$ are known for all of these simulated points $\widetilde{x}^{n,p,m}$ 

After that we proceed backward(Dynamic programming or Bellman approach), the function $v^{BM} (t, ·)$ is known for t = T since it is equal to the payoff function $\Psi (·)$ and thanks to the above result, it is known, through an approximation for the other t.

Unfortunately,$v^{BM}(t_{n+1}, \widetilde{x}^{n,p,m})$  is not known for all $t_n$, instead we will aproximate it over the set $ \widetilde{X}^{n,p}$ (our data) by means of GPR.

>Then we finally obtain the following expression: $ \hat{v}^{BM}(t_{n-1},x^{n-1,p}) = max(\Psi(x^{n-1,p}), \frac{e^{−r∆t}}{M} \sum_{m=1}^M v^{BM,GPR}_n( \widetilde{x}^{n-1,p,m}))$ 


#### GPR from scratch

In [18]:
def compute_gpr_parameters(K, K_star2, K_star, sigma_p = sigma_p ):
    """Compute gaussian regression parameters."""
    d = K.shape[0]
    # Weight.
    weight = np.dot(np.linalg.inv(K + (sigma_p**2)*np.eye(P)), y)
    # Mean.
    f_bar_star = np.dot(K_star, weight)
    # Covariance.
    cov_f_star = K_star2 - np.dot(K_star, np.dot(np.linalg.inv(K + (sigma_p**2)*np.eye(d)), K_star.T))
    
    return (f_bar_star, cov_f_star, weight)

In [19]:
def GPR_MC_Method(func, S0, strike_K):
    # gamma is the squarre root of Gamma the Correlation Matrix
    # we can proceed by a backward computation of v_BM_tilde 

    v_bm_dyn = np.array([func(x[N-1][p], strike_K) for p in range(P)])

    for n in range(N-2, 0, -1):
        for p in range(P):
            s = 0
            sigma_f_MLE, sigma_l_MLE,sigma_p_MLE = ComputeSigma_lfp(x_tilde[n][p], v_bm_dyn[p])
            for m in range(M):
                K, K_star2, K_star = compute_cov_matrices(x[n][p], x_tilde[n][p][m], sigma_f = sigma_f_MLE, l = sigma_l_MLE)
                # Compute posterior mean and covariance. 
                f_bar_star, cov_f_star, weight = compute_gpr_parameters(K, K_star2, K_star, sigma_p = sigma_p_MLE)
                s += f_bar_star

            v_bm_dyn[p] = np.maximum(func(x[n][p], K), s * np.exp(-r * delta_t)/M)
        
        s = 0
          for m in range(M):
            K, K_star2, K_star = compute_cov_matrices(S0, x_tilde[0][0][m], sigma_f=sigma_f, l=l)
            f_bar_star, cov_f_star, weight = compute_gpr_parameters(K, K_star2, K_star, sigma_p)
            s += f_bar_star

    return np.maximum(func(S0, K), s * np.exp(-r * delta_t)/M)

#### GPR from scikitlearn

This first method compute the Gaussian Process Regression over the data set (X,y) by using the SciKitlearn Library.

But, we also computed (see in the next section) the Gaussian Process Regression from scratch as explained in the article.

In [23]:
from sklearn.datasets import make_friedman2
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

def GPR_approximation(X,y):
    '''
    --------------------------------------------------------------------
    This function train the GPR model over the dataset (X,y).
    --------------------------------------------------------------------
    INPUTS:
    X: matrix, y: vector 
    RETURNS: GPR model
    --------------------------------------------------------------------
    '''
    sigma_l, sigma_f,sigma_p=ComputeSigma_lfp(X,y)
    kernel = (sigma_f**2)*RBF(sigma_l)
    
    gpr = GaussianProcessRegressor(kernel=kernel,
         random_state=0).fit(X, y)
    return gpr

#### GPR Monte Carlo Method Version 2
Here we use the scikit learn library to implement the GPR before computing the price

> Let us note that for most of the functions below, we use the vectorialisation trick of python to speed up our execution (The iterative version which of course are too slow while executing are commented)

#####  Choice of the set $X^n$
>here we use a deterministic space-filling sequence based on the Halton sequence

In [24]:
###############
def compute_x_n(S0,r,Neta, Sigma,gamma,tn,P): # For a fixed n
    """
    Compute set x_np for tn.
    Args:
        S0: spot price
        Neta: array of dividend
        r:interest rate
        Sigma:array of volatilities of assets
        gamma : square of the covariance matrix
        tn: time
        P: Cardinal of the set Xn
    Returns: X_n of shape (P,d)
    """
    d=S0.shape[0]
    sample=Halton_sequence_Rd(d,P)
    X_n=S0 * np.exp(r-Neta-0.5*(Sigma**2)*tn + np.sqrt(tn)*Sigma*(norm.ppf(sample, loc=10, scale=2)@gamma))
    return X_n

#####  Choice of the set $X^n_p$

In [25]:
def compute_X_tilde_np(x_np,r,Neta, Sigma,gamma,DeltaT,M): # For a fixed n and a fixed p
    """
    Compute the profit and loss.
    
    Args:
        x_np: shape (d,1)
    Returns:
       X_tilde_np: shape (M,d)
     
    """
    d=x_np.shape[0]
    G=np.random.multivariate_normal(np.zeros(d),np.identity(d),size=M)
    X_tilde_np = x_np * np.exp(r-Neta-0.5*(Sigma**2)*DeltaT + np.sqrt(DeltaT)*Sigma*(G@gamma))
    return X_tilde_np

#####  The GPR Monte Carlo Metho version 2

In [26]:
def GPR_MC_Method_Version2(func,S0,K,r,Neta, Sigma,gamma,N,T,P,M):
    # gamma is the squarre root of Gamma the Correlation Matrix
    # we can proceed by a backward computation of v_BM_tilde 
   
    times = np.linspace(0, T, N, endpoint=True)
    dt = T/N
    # first value n=N
    X_N=compute_x_n(S0,r,Neta, Sigma,gamma,T,P) # shape (P,d) ?
    v_BM=[]
    for p in range(P):
        v_BM.append(func(X_N[p],K)) 
    v_BM=np.array(v_BM) # shape(P,1)
    
    # second value n=N-1
    X_n=compute_x_n(S0,r,Neta, Sigma,gamma,times[N-1],P) # shape (P,d) ?
    
    v_BM_tilde=[] # the different approximation for each p
    for p in range(P):
        x_np=X_n[p]
        x_tilde_np=compute_X_tilde_np(x_np,r,Neta, Sigma,gamma,dt,M) # equivalent to S_tN (M,d)
        # MC
        v_BM=sum([func(x_tilde_np[m],K) for m in range(M)])/M
        v_BM_tilde.append(np.maximum(func(x_np,K),np.exp(-1*r*dt)*v_BM))
    v_BM_tilde=np.array(v_BM_tilde)
    
    # train a GPR
    gpr=GPR_approximation(X_n,v_BM_tilde)
        
    #intermadiate values
    for n in range(N-2,0,-1):
        X_n=compute_x_n(S0,r,Neta, Sigma,gamma,times[n],P) # (P,d)
        
        v_BM_tilde=[] # the different approximation for each p
        for p in range(P):
            x_np=X_n[p]
            x_tilde_np=compute_X_tilde_np(x_np,r,Neta, Sigma,gamma,dt,M) # equivalent to Stn+1(M,d)
            v_n=gpr.predict(x_tilde_np)
            v_BM_tilde.append(np.maximum(func(x_np,K),np.exp(-1*r*dt)*v_n.mean()))
        v_BM_tilde=np.array(v_BM_tilde) #.reshape(-1)
        # train a GPR
        # print(n)
        gpr=GPR_approximation(X_n,v_BM_tilde)

    # and then initial value:
    x_tilde_0=compute_X_tilde_np(S0,r,Neta, Sigma,gamma,dt,M) # shape(M,d)
    v_=gpr.predict(x_tilde_0)  
    v_BM_tilde_0 = np.maximum(func(S0,K),np.exp(-r*dt)*v_.mean())
  
    return v_BM_tilde_0

##### Let run an example:

In [39]:
def  Halton_sequence_Rd(d,Q):
    """output: h shape (d,q) 
    """
    sampler = qmc.Halton(d=d, scramble=False) # seed=None
    sample = sampler.random(n=Q+1)[1:]
    return sample

In [40]:
S0=np.array([100,100,100])
# K=100
d=3
sigma_i=0.2
rho=0.2
Neta=np.zeros(d)
Sigma=variance(sigma_i,d)

gamma,CovarMatrix=square_root_correlation_matrix(rho,d,sigma_i)
gamma

array([[0.99069011, 0.09626292, 0.09626292],
       [0.09626292, 0.99069011, 0.09626292],
       [0.09626292, 0.09626292, 0.99069011]])

In [49]:
T=2
N=10
P=250
M=100
K=110
func=Arithmetic_Basket_Put_Options #function.__Arithmetic_Basket_Put_Options__
val=GPR_MC_Method_Version2(Arithmetic_Basket_Put_Options,S0,K,r,Neta, Sigma,gamma,N=10,T=2,P=250,M=100)
print('Price of the american Arithmetic_Basket_Put_Options By GPR_MC for K=110:',val)

Price of the american Arithmetic_Basket_Put_Options By GPR_MC for K=110: 10.0


## The GPR Monte Carlo Control Variate 

>Let us present the GPR Monte Carlo Control Variate method (GPR-MC-CV), that is the proposed algorithm of the article.
The control variate technique is commonly used to reduce the variance of Monte Carlo estimators, but it can also give its contribution in American pricing.

>Let us consider an American and an European option with the same payoff function Ψ and maturity T , and let $v^{AM}$ , $v^{EU}$ denote their prices respectively. For a fixed time t and underlying stocks x, we define the American-European price gap
as: $v(t, x) = v^{AM}(t, x) − v^{EU}(t, x)$, then v (T, x) = 0.

> It is straightforward to see that $ v(t, x) = sup_{τ \in \mathbb{T}_{t,T}} \mathbb{E}_{t,x}[e^{−r(τ−t)}\hat{Ψ}(τ, Sτ )]$ where $T_t$,T stands for the set of all stopping times taking values in $[t, T ]$ and Ψ is defined by $\hat{Ψ} (t, x) = Ψ (x) − v^{EU}(t, x)$.

Let us consider a set $Z = {z^q, q = 1, . . . , Q}$ consisting of Q points in $R^d$ quasi-randomly distributed quasi-randomly distributed
according to the law of the vector $ (σ_1W_1^1, . . . , σ_dW_d^d)^T$

In particular, we define $z^q_i =\sqrt{T}σ_iΣ_ih^q$
where $Σ_i$ is i-th row of the matrix Σ and is i-th row of the matrix Σ and $h^q$
is the q-th point of the Halton sequence in $R^d$

In [43]:
def compute_Z(T,Sigma,gamma,Q):
    """ Sigma: (d,1)
         Covariance: (d,d)
    output: Z shape (Q,d) 
    """
    Z=[]
    H = sample
    Z=np.sqrt(T)*Sigma*(norm.ppf(H, loc=10, scale=2)@gamma)
    return Z

In [44]:
# Helper function to calculate the respective covariance matrices
def cov_matrix(x1, x2, cov_function) -> np.array:
    return np.array([[cov_function(a, b) for a in x1] for b in x2])

def compute_Z(T,Sigma,gamma,Q):
    """ Sigma: (d,1)
         Covariance: (d,d)
    output: Z shape (d,q) 
    """
    Z=[]
    for q in range(Q):
        z=np.sqrt(T)*Sigma*(gamma@norm.ppf(sample[q], loc=10, scale=2))
        Z.append(Z)
    return np.array(Z)

In a nutshell, the main idea is to approximate the function u by training the GPR method on the set Z.
> we compute ω using the formular: $ w = (K(X,X) + σ^2_P I_P)^{−1}y$
By cholesky decomposition, we will have: $(K(X,X) + σ^2_P I_P)^{−1}=(LL^T)^{−1}=L^{-T}L^{-1}$ and then $ w =L^{-T}L^{-1}y$, if we set $L^{-1}y=m$,  to find w, we just need to solve the equation $L^Tw=y$

In [45]:
def approximation_u(func,S0,K,T,r,Neta,Sigma,gamma,Q):
    y=[]
    Z=compute_Z(T,Sigma,gamma,Q)
    for q in range(Q):
        z=Z[q]
        ST=S0*np.exp((r-Neta-0.5*(Sigma**2))*T+z)
        y.append(func(ST,K))
    y=np.array(y)
    sigma_f_MLE, sigma_l_MLE,sigma_p_MLE= ComputeSigma_lfp(Z,y)
    K= Compute_K(Z,sigma_f_MLE,sigma_l_MLE)  
    L = np.linalg.cholesky(K + sigma_p_MLE*np.eye(Q))
    m = np.linalg.solve(L, y)
    w=np.linalg.solve(L.T, m)
    return w

####  Machine Learning Exact Integration for European options

 Let $u : Z \rightarrow R$ be the function
defined by
$u(z) := \Psi((S_0 exp((r − \eta- \frac{1}{2} \sigma^2)T + z))$

In a nutshell, the main idea is to approximate the function u by training the GPR method on the set Z.
In particular, we employ the Squared Exponential kernel and we then
approximate the function u(·) by
$u^{GPR}(z) = \sum_{q=1}^Q k_{SE}(z_q, z) ω_q$
where $ω_1, . . . , ω_Q$ are weights of the approximation.

> we compute ω using the formular: $ w = (K(X,X) + σ^2_P I_P)^{−1}y$

This can be done quickly by cholesky decomposition: we will have: $(K(X,X) + σ^2_P I_P)^{−1}=(LL^T)^{−1}=L^{-T}L^{-1}$ and then $ w =L^{-T}L^{-1}y$, if we set $L^{-1}y=m$,  to find w, we just need to solve the equation $L^Tw=y$

In [46]:
def approximation_u(func,S0,K,T,r,Neta,Sigma,gamma,Q):
    '''
    --------------------------------------------------------------------
    This function computes the weights of the GPR approximation model.
    --------------------------------------------------------------------
    INPUTS:
    func: the basket option function
    Sigma: vector of varaiance
    gamma: squared root of the correlation matrix
    RETURNS: w
    --------------------------------------------------------------------
    '''
    y = []
    Z = compute_Z(T,Sigma,gamma,Q)
    matrix_S = S0*np.exp((r-Neta-0.5*(Sigma**2))*T+Z)
    y = np.array([func(matrix_S[i],K) for i in range(Q)]) # Q=matrix_S.shape[0]
    sigma_f_MLE,sigma_l_MLE,sigma_p_MLE= ComputeSigma_lfp(Z,y)
    K = Compute_K(Z,sigma_f_MLE,sigma_l_MLE)
    # w = np.linalg.inv(K + sigma_p_MLE*np.eye(Q))@y  
    L = np.linalg.cholesky(K + sigma_p_MLE*np.eye(Q))
    m = np.linalg.solve(L, y)
    w = np.linalg.solve(L.T, m)

    return w,sigma_f_MLE,sigma_l_MLE,Z

Below, the function pricing_EU_option is the  Machine Learning Exact Integration for European options used for controle varaite.
The function phi_tilde is used to compute the american option

In [ ]:
def pricing_EU_option(w,S0,x,Z,Q,CovarMatrix,r,Neta,t,T,sigma_f,sigma_l):
    '''
    --------------------------------------------------------------------
    This function computes theprice of the european option using GPR on the set Z.
    --------------------------------------------------------------------
    INPUTS:
    w: weights vector
    RETURNS: european price
    --------------------------------------------------------------------
    '''
    sum_=0
    z_tilde=np.log(x/S0)-(r-Neta-0.5*(Sigma**2))*t
    P = (T-t)*CovarMatrix+(sigma_l**2)*np.eye(d)
    L = np.linalg.cholesky(P)

    for q in range(Q):
        z = Z[q]
        m = np.linalg.solve(L, (z-z_tilde))
        factor = np.linalg.solve(L.T, m)
        sum_+= w[q]*np.exp(-0.5*(z-z_tilde).T @ factor)
    
    v = (sigma_f**2)*(sigma_l**d)*sum_ / (np.linalg.det(L))**2
    v_EU = np.exp(-r*(T-t))*v
    return v_EU

    

def phi_tilde(func,w,S0,x,K,Z,Q,CovarMatrix,r,Neta,t,T,sigma_f,sigma_l):
    return func(x,K) - pricing_EU_option(w,S0,x,Z,Q,CovarMatrix,r,Neta,t,T,sigma_f,sigma_l)


In [47]:
def GPR_MC_CV_algorithm(func,S0,K,r,Neta, Sigma,gamma,CovarMatrix,N,T,P,Q,M):
    '''
    --------------------------------------------------------------------
    This function computes the price of the american/bermuda option by controle variate.
    --------------------------------------------------------------------
    INPUTS:
    func: the basket option function
    RETURNS: the price at initial time
    --------------------------------------------------------------------
    '''
    # gamma is the squarre root of Gamma the Correlation Matrix
    # we can proceed by a backward computation of v_BM_tilde 
    
    times = np.linspace(0, T, N, endpoint=True)
    dt = T/N
    w,sigma_f,sigma_l,Z =approximation_u(func,S0,K,T,r,Neta,Sigma,gamma,Q)
    
    # first value n=N useless
    """
    X_np= compute_x_n(S0,r,Neta, Sigma,gamma,T,P)
    X_tilde_np=compute_X_tilde_np(x_np,r,Neta, Sigma,gamma,dt,M)
    v_EU=pricing_EU_option(w,P,r,t,T,d) # for each tn ?
    """

    ###################
    # second value n=N-1
    X_n=compute_x_n(S0,r,Neta, Sigma,gamma,times[N-1],P) # shape (P,d) ?
   # the different approximation for each p 
    v_BM_tilde=np.array([phi_tilde(func,w,S0,X_n[p],K,Z,Q,CovarMatrix,r,Neta,times[N-1],T,sigma_f,sigma_l) for p in range(P)])

    # train a GPR
    gpr=GPR_approximation(X_n,v_BM_tilde)
            
    for n in range (N-2,0,-1):
        #Step n:
        X_n=compute_x_n(S0,r,Neta, Sigma,gamma,times[n],P) # (P,d)
        v_tilde=[] 
        for p in range(P):
            x_np=X_n[p]
            x_tilde_np=compute_X_tilde_np(x_np,r,Neta, Sigma,gamma,dt,M) # equivalent to Stn+1(M,d)
            v_n=gpr.predict(x_tilde_np)
        
            phi_=phi_tilde(func,w,S0,x_np,K,Z,Q,CovarMatrix,r,Neta,times[n],T,sigma_f,sigma_l)
            right= np.exp(-r*dt) * v_n.mean()
            v_tilde.append(np.maximum(phi_,right))
        v_tilde=np.array(v_tilde) #.reshape(-1)
        # train a GPR
        # print(n)
        gpr=GPR_approximation(X_n,v_tilde)

        
    #step 0
    x_tilde_0=compute_X_tilde_np(S0,r,Neta, Sigma,gamma,dt,M) # shape(M,d)
    v_=gpr.predict(x_tilde_0)  
    right= np.exp(-r*dt) *v_.mean()
    v_tilde_0 = np.maximum(func(S0,K),right)  # func or phi_tilde(func,times[0],x_np,w,K) ?
    # t=0, spot=S0
    v_BM=v_tilde_0+pricing_EU_option(w,S0,S0,Z,Q,CovarMatrix,r,Neta,0,T,sigma_f,sigma_l)
    return v_BM

##### Example:


In [ ]:
T=2
N=10
P=250
M=100
K=110
func=Arithmetic_Basket_Put_Options
val2=GPR_MC_CV_algorithm(func,S0,K,r,Neta, Sigma,gamma,CovarMatrix,N,T,P,Q,M)
print('Price of the american Arithmetic_Basket_Put_Options By GPR_MC for K=110:',val2)

8
7
6
5
4
3
2
1
Price of the american Arithmetic_Basket_Put_Options By GPR_MC for K=110: 10.0
